# Examining and thresholding sensitivity of a probe to the cortex using the Schaefer parcellation scheme

This notebook shows how to examine the theoretical sensitivity of a probe on a headmodel to brain areas (here we use parcel coordinates from the Schaefer 2018 atlas), and how to identify parcels that should be dropped, because changes in them cannot be observed. For this the original designed probe can also be reduced to an effective probe by dropping channels that are pruned due to bad signal quality.

In [1]:
# set this flag to True to enable interactive 3D plots
INTERACTIVE_PLOTS = True

In [ ]:
import pyvista as pv

import cedalion.sigproc

if INTERACTIVE_PLOTS:
    pv.set_jupyter_backend('html')
else:
    pv.set_jupyter_backend('static')

import os

import matplotlib.pyplot as p
import numpy as np
import xarray as xr
import pint
from IPython.display import Image

import cedalion
import cedalion.dataclasses as cdc
import cedalion_parcellation.datasets
import cedalion_parcellation.imagereco.forward_model as fw
import cedalion.imagereco.tissue_properties
import cedalion.sigproc.quality as quality
import cedalion.io
import cedalion.plots
import cedalion_parcellation.plots
from cedalion.vis import plot_sensitivity_matrix
from cedalion.imagereco.solver import pseudo_inverse_stacked
from cedalion import units

xr.set_options(display_expand_data=False)

## Load a DOT finger-tapping dataset
and perform some very basic quality checks to identify bad channels

In [3]:
# load example dataset
rec = cedalion_parcellation.datasets.get_fingertappingDOT()

# check signal quality using a simple SNR threshold
snr_thresh = 30 # the SNR (std/mean) of a channel. Set very high here for demonstration purposes

# SNR thresholding using the "snr" function of the quality subpackage
snr, snr_mask = quality.snr(rec["amp"], snr_thresh)

# prints all channels in snr_mask that are "False"
print("Channels with SNR below threshold:")
print(np.where(~snr_mask)[0])

Channels with SNR below threshold:
[ 0  2  3  4  5  8  9 11 12 19 23 29 31 32 51 52 53 54 55 56 58 59 61 62
 64 69 70 73 74 79 81 82 84 85 86 87 95]


## Load a headmodel and precalulated fluence profile

In [4]:
# load pathes to segmentation data for the icbm-152 atlas
SEG_DATADIR, mask_files, landmarks_file = cedalion_parcellation.datasets.get_icbm152_segmentation()
PARCEL_DIR = cedalion_parcellation.datasets.get_icbm152_parcel_file()

# create forward model class for icbm152 atlas
head = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR,
    mask_files = mask_files,
    brain_surface_file= os.path.join(SEG_DATADIR, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file,
    parcel_file=PARCEL_DIR,
    brain_face_count=None,
    scalp_face_count=None
)

# snap probe to head and create forward model
geo3D_snapped = head.align_and_snap_to_scalp(rec.geo3d)
fwm = fw.ForwardModel(head, geo3D_snapped, rec._measurement_lists["amp"])


load precomputed fluce, calculate sensitivity on the cortex and plot it on head model

In [6]:
# load precomputed fluence for this dataset and headmodel
fluence_all, fluence_at_optodes = cedalion_parcellation.datasets.get_precomputed_fluence("fingertappingDOT", "icbm152")

# calculate Adot sensitivity matrix
Adot = fwm.compute_sensitivity(fluence_all, fluence_at_optodes)

# plot on head model
plotter = plot_sensitivity_matrix.Main(
    sensitivity=Adot,
    brain_surface=head.brain,
    head_surface=head.scalp,
    labeled_points=geo3D_snapped,
)
plotter.plot(high_th=0, low_th=-3)
plotter.plt.show()

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Investigation of Parcels and effective Parcel Sensitivity
First plot full parcellation scheme on head

In [ ]:
b = cdc.VTKSurface.from_trimeshsurface(head.brain)
b = pv.wrap(b.mesh)
b["parcels"] = parcels.Color.tolist()

plt = pv.Plotter()

plt.add_mesh(
    b,
    scalars="parcels",
    rgb=True
)


if "parcel" in head.brain.vertices.coords:
    cog = head.brain.vertices.mean(["label", "parcel"]).values
else:
    cog = head.brain.vertices.mean("label").values
plt.camera.position = cog + [0,0,400]
plt.camera.focal_point = cog 
plt.camera.up = [0,1,0] 
plt.reset_camera()

plt.show()

TrimeshSurface(mesh=<trimesh.Trimesh(vertices.shape=(15002, 3), faces.shape=(29978, 3))>, crs='ijk', units=<Unit('dimensionless')>, _vertices=<xarray.DataArray (parcel: 15002, label: 15002, ijk: 3)> Size: 5GB
[] 79.85 27.23 154.7 78.43 27.11 151.0 ... 105.5 161.7 213.3 120.0 84.61 208.7
Coordinates:
  * label    (label) int32 60kB 0 1 2 3 4 5 ... 14997 14998 14999 15000 15001
  * parcel   (parcel) <U44 3MB 'VisCent_ExStr_11_LH' ... 'DorsAttnB_PostC_11_RH'
Dimensions without coordinates: ijk)

c:\Users\avonl\anaconda3\envs\cedalion\Lib\site-packages\xarray\core\variable.py:338: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  data = np.asarray(data)


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [55]:
#display(head.brain)
display(Adot)

<xarray.DataArray (channel: 100, vertex: 25020, wavelength: 2)> Size: 40MB
2.075e-20 2.075e-20 6.024e-22 6.024e-22 4.449e-22 ... 0.0 0.0 0.0 0.0 0.0
Coordinates:
  * channel     (channel) <U6 2kB 'S1D1' 'S1D2' 'S1D4' ... 'S14D31' 'S14D32'
  * wavelength  (wavelength) float64 16B 760.0 850.0
    is_brain    (vertex) bool 25kB True True True True ... False False False
    parcel      (vertex) <U44 4MB 'VisCent_ExStr_11_LH' ... 'scalp'
Dimensions without coordinates: vertex

In [ ]:
chan_mask = snr_mask
dOD_thresh = 0.01
minCh = 2
dHbO = 0.1
dHbR = 0.03
parcels = cedalion_parcellation.io.read_parcellations(PARCEL_DIR)

# copies Adot and keeps only those vertices whose is_brain coordinate is true
Adot_brain = Adot.sel(vertex=Adot.coords['is_brain'])

assert "wavelength" in Adot_brain.dims, "no wavelength dimension in Adot"  # FIXME move to validate schema
wavelengths = Adot_brain.wavelength.values
assert len(wavelengths) == 2, "expected two wavelengths in Adot" 

assert (
    len(chan_mask) == len(Adot_brain.channel)
), "number of channels in chan_mask and Adot do not match"

# get extinction coefficients
ec = cedalion.nirs.get_extinction_coefficients("prahl", wavelengths)

# set up xarray with chromophore changes according to user input
dHb = xr.DataArray(
    [dHbO*1e-6, dHbR*1e-6],
    dims=["chromo"],
    coords={"chromo": ["HbO", "HbR"]},
    attrs={"units": "M"},
    )
dHb = dHb.pint.quantify()

# sum in Adot over all vertices that belong to the same parcel, making Adot #ch x #parcels x #wavelength
Adot_bparcel = Adot_brain.groupby("parcel").sum("vertex")

# Calculate dOD = Adot * exctinciton_coefficients * deltaHb 
Adot_ec = xr.dot(Adot_bparcel, ec)
dOD = xr.dot(Adot_ec, dHb)

#display(Adot)
display(ec)
display(Adot_bparcel)
display(Adot_ec)
display(dOD)



#set chromophore changes
#dHb = xr.DataArray([dHbO, dHbR], dims="chromo", coords={"chromo" : ["HbO", "HbR"]})

#conc.pint.to("micromolar")


<xarray.DataArray (channel: 100, vertex: 25020, wavelength: 2)> Size: 40MB
2.075e-20 2.075e-20 6.024e-22 6.024e-22 4.449e-22 ... 0.0 0.0 0.0 0.0 0.0
Coordinates:
  * channel     (channel) <U6 2kB 'S1D1' 'S1D2' 'S1D4' ... 'S14D31' 'S14D32'
  * wavelength  (wavelength) float64 16B 760.0 850.0
    is_brain    (vertex) bool 25kB True True True True ... False False False
    parcel      (vertex) <U44 4MB 'VisCent_ExStr_11_LH' ... 'scalp'
Dimensions without coordinates: vertex

Magnitude,[[134.93148644945109 243.61350283877005] [356.559906820314 159.1823126488644]]
Units,1/(millimeter molar)


<xarray.DataArray (channel: 100, parcel: 602, wavelength: 2)> Size: 963kB
1.052e-05 1.052e-05 1.256e-09 1.256e-09 ... 2.494e-10 3.247e-10 3.247e-10
Coordinates:
  * channel     (channel) <U6 2kB 'S1D1' 'S1D2' 'S1D4' ... 'S14D31' 'S14D32'
  * wavelength  (wavelength) float64 16B 760.0 850.0
  * parcel      (parcel) object 5kB 'Background+FreeSurfer_Defined_Medial_Wal...

Magnitude,[[[0.003982113036121457 0.005425362568629127] [4.753978137574166e-07 6.476977123883285e-07] [3.895450221991394e-08 5.307290282983701e-08] ... [5.244419889052716e-13 7.145171194827152e-13] [2.1360875551810964e-14 2.9102763683679e-14] [1.3422699229868583e-14 1.828752958821706e-14]] [[4.919686729090546e-05 6.702743992266527e-05] [2.3060447796256213e-08 3.141831714839813e-08] [1.4832882046365727e-07 2.020880932039648e-07] ... [2.1821149906658403e-13 2.9729856695212747e-13] [9.858363619217879e-15 1.3431360808314462e-14] [1.096859308817356e-13 1.4943974174339824e-13]] [[0.0002443495772059196 0.0003329099495187999] [2.3206635554048056e-08 3.1617488186970196e-08] [1.4939063777377817e-08 2.0353474824283415e-08] ... [5.165396388586987e-13 7.037506962903089e-13] [1.2709213971366062e-14 1.7315453662788665e-14] [5.4862890991951e-15 7.474701810183184e-15]] ... [[3.063078439573215e-08 4.173239423414641e-08] [7.872433878949959e-08 1.0725664415710023e-07] [1.5468505266208425e-08 2.1074803427897982e-08] ... [7.400698879630712e-09 1.0082957042914876e-08] [5.553131616690496e-08 7.565770267839441e-08] [3.3466234335539885e-08 4.5595505057247964e-08]] [[5.1980052017963075e-09 7.0819342890458886e-09] [2.5921137547107374e-07 3.5315815525251206e-07] [2.007327907823056e-09 2.7348499641474416e-09] ... [2.8033988935293037e-09 3.819443417081919e-09] [1.0765858931077055e-06 1.4667762450233355e-06] [1.2997696293452046e-08 1.7708491524285664e-08]] [[6.428354488546931e-08 8.758202869602774e-08] [9.23530688542485e-08 1.2582487697232358e-07] [2.205395541677676e-08 3.0047038625737996e-08] ... [2.915324210539107e-08 3.9719341725875385e-08] [9.441115199741245e-08 1.2862887754859143e-07] [1.2292600450208758e-07 1.67478456158124e-07]]]
Units,1/(millimeter molar)


Magnitude,[[5.609721806710195e-10 6.697071274739151e-14 5.487637306886504e-15 ... 7.387971247500861e-20 3.009170465691466e-21 1.89089581063337e-21] [6.930509926770504e-12 3.2485942940775654e-15 2.0895524842484668e-14 ... 3.0740106915222226e-20 1.3887771861712217e-21 1.5451785340475505e-20] [3.4422256206155955e-11 3.269188201013911e-15 2.1045106224662843e-15 ... 7.276648477457914e-20 1.7903850070202662e-21 7.728699642250055e-22] ... [4.315050266597607e-15 1.1090133203662966e-14 2.179094629457782e-15 ... 1.0425585992505174e-15 7.822862697042328e-15 4.714488585271427e-15] [7.322585488510074e-16 3.6515882204682737e-14 2.8277828970672887e-16 ... 3.949231918653879e-16 1.5166187666147062e-13 1.8310243750737746e-15] [9.055815349427762e-15 1.3010053194594557e-14 3.106806700449816e-15 ... 4.106904462315368e-15 1.3299981526198987e-14 1.7316954134952475e-14]]
Units,1/millimeter
